In [1]:
%%time

import os
import boto3
import sagemaker

from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

CPU times: user 1.02 s, sys: 173 ms, total: 1.2 s
Wall time: 1.43 s


In [3]:
bucket = "model-migration"
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

In [4]:
print(role)
print(region)
print(bucket)
print(bucket_path)

arn:aws:iam::327437885319:role/service-role/AmazonSageMaker-ExecutionRole-20230120T155247
us-east-1
model-migration
https://s3-us-east-1.amazonaws.com/model-migration


In [5]:
import sklearn
print(sklearn.__version__)

1.0.1


In [6]:
#pip install scikit-learn==1.0.1

In [7]:
model_file_name = "titanic_GB_model.pkl"

In [8]:
import joblib
import sklearn

model = joblib.load(model_file_name)

In [9]:
model

GradientBoostingClassifier()

In [10]:
!tar czvf titanic_GB_model.tar.gz $model_file_name

titanic_GB_model.pkl


In [11]:
#### prefix in S3
prefix = "models/titanic"

fObj = open("titanic_GB_model.tar.gz", "rb")
key = os.path.join(prefix,"titanic_GB_model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

models/titanic/titanic_GB_model.tar.gz


In [12]:
model_data = "s3://{}/{}".format(bucket, key)
print(f"model data: {model_data}")

model data: s3://model-migration/models/titanic/titanic_GB_model.tar.gz


In [13]:
#scaler = "dg_scaler_v5"
#scaler = joblib.load(scaler)
#scaler

In [14]:
%%writefile script.py

import joblib
import os
from io import StringIO
import numpy as np
import pandas as pd
import sklearn

def predict_fn(input_object, model):
    #prediction = model.predict(input_object)
    pred_prob = model.predict_proba(input_object)
    return pred_prob

def model_fn(model_dir):
    print("loading model.joblib from: {}".format(model_dir))
    loaded_model = joblib.load(os.path.join(model_dir, "titanic_GB_model.pkl"))
    return loaded_model


Writing script.py


In [15]:
from sagemaker.sklearn.model import SKLearnModel
model = SKLearnModel(
    role=role,
    model_data=model_data,
    framework_version="1.0-1",
    py_version="py3",
    entry_point="script.py",
)

Create a Model Package Group (Boto3):

In [16]:
import time
import os
from sagemaker import get_execution_role, session
import boto3



sm_client = boto3.client('sagemaker', region_name=region)

Create a Model Package Group for the trained model to be registered:

In [17]:
import time
client = boto3.client("sagemaker")

model_package_group_name = "sample-titanic-models"
model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": "GB model",
}

create_model_pacakge_group_response = client.create_model_package_group(
    **model_package_group_input_dict
)
model_package_arn = create_model_pacakge_group_response["ModelPackageGroupArn"]
print(f"ModelPackageGroup Arn : {model_package_arn}")

ClientError: An error occurred (ValidationException) when calling the CreateModelPackageGroup operation: Model Package Group already exists: arn:aws:sagemaker:us-east-1:327437885319:model-package-group/sample-titanic-models

Register the model of the 1st training job in the Model Registry:

In [20]:
inference_instance_type = "ml.m5.xlarge"
model_package_arn = "arn:aws:sagemaker:us-east-1:327437885319:model-package-group/sample-titanic-models"
model_package_1 = model.register(
    model_package_group_name="sample-titanic-models",
    inference_instances=[inference_instance_type],
    transform_instances=[inference_instance_type],
    content_types=["text/csv"],
    response_types=["text/csv"],
    approval_status="Approved",
    description = 'titanic_GB_model'
)

model_package_arn_1 = model_package_1.model_package_arn
print("Model Package ARN : ", model_package_arn_1)

Model Package ARN :  arn:aws:sagemaker:us-east-1:327437885319:model-package/sample-titanic-models/1


In [21]:
model_package_arn = "arn:aws:sagemaker:us-east-1:327437885319:model-package/sample-titanic-models/1"
from sagemaker import ModelPackage 
from time import gmtime, strftime
sess = sagemaker.Session()

titanic_model = ModelPackage(role=role, 
                     model_package_arn=model_package_arn, 
                     sagemaker_session=sess)
sm_model = titanic_model.transformer(
    instance_count=1,
    instance_type="ml.m5.xlarge",
   
)